# Zadania

In [193]:
# Two-class MNIST 
import os
from keras.datasets import mnist
import numpy as np
import random

d1 = 5
d2 = 6

(mnist_x_train, mnist_y_train), (mnist_x_test, mnist_y_test) = mnist.load_data()

X_train = (mnist_x_train.astype('float32') / 255.).reshape((len(mnist_x_train), np.prod(mnist_x_train.shape[1:])))
y_train = mnist_y_train
X_test = (mnist_x_test.astype('float32') / 255.).reshape((len(mnist_x_test), np.prod(mnist_x_test.shape[1:])))
y_test = mnist_y_test

X_train = X_train[np.logical_or(y_train == d1, y_train == d2)]
y_train = y_train[np.logical_or(y_train == d1, y_train == d2)]
y_train[y_train==d1] = 0
y_train[y_train==d2] = 1
X_test = X_test[np.logical_or(y_test == d1, y_test == d2)]
y_test = y_test[np.logical_or(y_test == d1, y_test == d2)]
y_test[y_test==d1] = 0
y_test[y_test==d2] = 1

print X_train.shape
print X_test.shape

(11339L, 784L)
(1850L, 784L)


## Ćwiczenie 1 [5 pkt]

Uzupełnij metody forward_pass oraz backward_pass w klasach ReLU, Sigmoid i Dense. Metoda forward_pass ma przyjmować batch inputów i zwracać batch outputów. Metoda backward_pass ma przyjmować batch inputów oraz batch pochodnych cząstkowych outputów i zwracać batch pochodnych cząstkowych inputów oraz wektor (**nie batch**) pochodnych cząstkowych wag. Jeśli wagi przechowujemy w macierzy dwuwymiarowej, to możemy najpierw policzyć pochodne cząstkowe w macierzy o takim samym kształcie, a następnie np. użyć .flat.

## Ćwiczenie 2 [4 pkt]

Uzupełnij metodę _forward_pass klasy Network. Metoda ta ma przyjmować batch inputów (X) i zwracać dwie rzeczy:
* inps - lista batchów inputów dla każdej warstwy w sieci (włącznie z X); te wartości będziemy używali w metodzie _backward_pass
* output - batch outputów z sieci (czyli $\mathbf{\hat y}$); output **nie** powinien być ostatnim elementem inps.

## Ćwiczenie 3 [5 pkt]

Uzupełnij metodę _backward_pass klasy Network. Zwróć uwagę, że pochodna funkcji kosztu po neuronach ostatniej warstwy jest już liczona w metodzie _fit_on_batch. Metoda ma zwracać listę layer_grads, której elementy to wektory pochodnych cząstkowych funkcji kosztu po kolejnych warstwach (zwrócone przez metodę Layer.backward_pass). Kolejność wektorów w tej liście ma być zgodna z kolejnością warstw w sieci.

## Ćwiczenie 4 [3 pkt]
Naucz sieć neuronową z jedną warstwą ukrytą i aktywacją Sigmoid na powyższych danych (dwuklasowy MNIST). Użyj MSE jako funkcji kosztu (oznacza to regresję do numeru klasy, co jest złym pomysłem, ale póki nie mamy klasy Crossentropy musi nam to wystarczyć). Użyj GD. Reportuj loss oraz accuracy.

## Ćwiczenie 5 [3 pkt]
Uzupełnić klasę Crossentropy, wzorując się na klasie MSE.

## Ćwiczenie  6 [3 pkt]
Uzupełnić klasę Momentum, wzorując się na klasie GD. Wzory można znaleźć tutaj: http://distill.pub/2017/momentum/

## Ćwiczenie 7 [3 pkt]
Naucz sieć neuronową z jedną warstwą ukrytą. Rozważ dwa przypadki: aktywację ReLU oraz Sigmoid. Czy jest sens używać ReLU jako ostatnią warstwę? Użyj Crossentropy jako funkcji kosztu. Użyj Momentum. Reportuj loss oraz accuracy.

## Ćwiczenie 8 [6 pkt]
Vanishing gradient.

Zadanie polega na zbadaniu zjawiska *vanishing gradient* w głębokich sieciach. Należy zmodyfikować kod warstwy Dense i dodać monitorowanie **normy euklidesowej** wektora delta_weights. Każdą warstwę Dense w trenowanej sieci należy monitorować oddzielnie. Po każdym wywołaniu metody fit_on_batch każdy z monitorów powinien zapamiętać nową normę. Po nauczeniu sieci dla każdej warstwy należy narysować wykres: poziomo - numer wywołania fit_on_batch, pionowo - norma delta_weights. Im niżej znajduje się warstwa Dense, tym silniej będzie zachodziło zjawisko *vanishing gradient*.

Naucz dwuwarstwową sieć z aktywacjami Sigmoid, reportując normy delta_weights. Powtórz to dla głębszej sieci (np. 6-10 warstw).

## Ćwiczenie 9 [4 pkt]
Przetestować kod z ćwiczenia 7. (dwuwarstwowa sieć) stosując inne inicjalizacje wag w warstwach Dense. Napisać własną inicjalizację wag, która sprawi, że sieć niczego się nie nauczy (init='stupid').


In [194]:
import math
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [195]:
class Layer():

    def forward_pass(self, input):
        # return output
        raise NotImplementedError()

    def backward_pass(self, input, output_grad):
        # return input_grad, weight_grad
        raise NotImplementedError()

    def update_weights(self, delta_weights):
        pass

    def debug_grad(self, evaluate_loss):
        return None

In [196]:
class ReLU(Layer):
    #Ćwiczenie 1 
    def forward_pass(self, input):
        input[input<=0]=0
        return input
    #Ćwiczenie 1 
    def backward_pass(self, input, output_grad):
        input[input<=0]=0
        input[input>0]=1
        tmp = np.vstack([ np.multiply( output_grad[i,:],   input[i,:]   )
                          for i in xrange(input.shape[0])])
        return  np.asmatrix(tmp), None    

In [197]:
from scipy.special import expit

def sigmoid(gamma):
    if gamma < -15: 
        return 1. - 1./(1. + math.exp(-15))
    if gamma > 15: 
        return 1./(1. + math.exp(-15.0))
    
    if gamma < 0:
        return 1. - 1./(1. + math.exp(gamma))
    else:
        return 1./(1. + math.exp(-gamma))

class Sigmoid(Layer):
    #Ćwiczenie 1 
    def forward_pass(self, input):
        f = np.vectorize(lambda x: sigmoid(x) , otypes=[np.float])
        return f(input)
    
    #Ćwiczenie 1 
    def backward_pass(self, input, output_grad):
        
        f = np.vectorize(lambda x: sigmoid(x) * sigmoid(-x) , otypes=[np.float])
        tmp = np.vstack([ np.multiply( output_grad[i],  
                    f(input[i,:])   )  for i in xrange(input.shape[0])])
    
        input_grad = np.asmatrix(tmp)
        weight_grad = None
        return  input_grad, weight_grad

In [198]:
class Dense(Layer):

    def __init__(self, input_size, output_size, init = 'gaussian', plot=False):
        self.plot = plot
        self.monitor = []
        input_size += 1
        if init == 'zeros':
            self.weights = np.zeros((input_size, output_size))
        elif init == 'gaussian':
            self.weights = np.random.normal(
                0.,
                2. / (input_size + output_size),
                (input_size, output_size)
            )
            
        elif init == 'aaa':
            self.weights = np.random.choice([0.0001, 1000000.01],  (input_size, output_size)  )
        else:
            raise NotImplementedError()
        self.weights = np.asmatrix(self.weights)
        
    #Ćwiczenie 1 
    def forward_pass(self, input):
        input = np.append(np.array([[1] for i in input]), input, axis=1)
        wynik = np.dot(input, self.weights)
        return wynik
    
    #Ćwiczenie 1 
    def backward_pass(self, input, output_grad):
        input = np.append(np.array([[1] for i in input]),input, axis=1)
        input_grad = np.asmatrix(np.dot(output_grad, (self.weights[1:,:]).T))
        weight_grad = np.asmatrix(np.dot(input.T,output_grad))
        return input_grad, weight_grad
        
    def update_weights(self, delta_weights):
        self.weights += delta_weights
        self.monitor.append(np.linalg.norm(delta_weights))

In [199]:
# Optimizers
class Optimizer():

    def calculate_deltas(self, grad):
        raise NotImplementedError()

class GD(Optimizer):

    def __init__(self, learning_rate):
        self.learning_rate = learning_rate

    def calculate_deltas(self, grad):
        return -self.learning_rate * grad      

In [171]:
#Ćwiczenie 6
class Momentum(Optimizer):

    def __init__(self, alpha, beta):
        self.alpha = alpha
        self.beta = beta
        self.remember = 0
    def calculate_deltas(self, grad):
        self.remember = self.beta * self.remember + grad
        return -self.alpha * self.remember

In [226]:
class Loss():

    def forward_pass(self, y, t):
        # return cost
        raise NotImplementedError()

    def backward_pass(self, y, t):
        # return y_grad
        raise NotImplementedError()

class MSE(Loss):

    def forward_pass(self, y, t):
        wynik = np.average(0.5 * np.square(y - t))
        return wynik 

    def backward_pass(self, y, t):
        return (y - t) / y.size

EPS = 0.000001
#Ćwiczenie 5
class Crossentropy(Loss):
    def forward_pass(self, y, t):
        wynik = np.average(-1.0*np.multiply(t, np.log(y))
                          -np.multiply(1.0 - t, np.log( 1.0 - y )) )
        return wynik
        
    def backward_pass(self, y, t):
        yy  = np.copy(y)
        tt  = np.copy(t)
        yy[yy<EPS]=EPS
        #yy[yy>1.0-EPS]=1.0-EPS
        tt[tt<EPS]=EPS
        #tt[tt>1.0-EPS]=1.0-EPS
        return (-np.divide(tt,yy)+np.divide((1-tt),(1-yy)))/ yy.size


In [227]:
class Network():
    def __init__(self, loss, optimizer, metrics = []):
        self.layers = []
        self.loss = loss
        self.optimizer = optimizer
        self.metrics = metrics

    def add(self, layer):
        self.layers.append(layer)

    def fit(self, X, t, epochs, batch_size=256, print_stats=False,plot=False):
        X = np.array(X)
        t = np.array(t)
        X = X.reshape(len(X), -1)
        t = t.reshape(len(t), -1)
        
        for epoch in range(epochs):
            if print_stats:
                print("Epoch %d" % (epoch+1))
                print("    -> batch size: %d" % batch_size)
            rng_state = np.random.get_state()
            np.random.shuffle(X)
            np.random.set_state(rng_state)
            np.random.shuffle(t)
            pos = 0
            while pos < len(X):
                batch_X = X[pos:pos+batch_size]
                batch_t = t[pos:pos+batch_size]
                self._fit_on_batch(batch_X, batch_t)
        
                pos += batch_size
            if print_stats:
                _, y = self._forward_pass(X)
                l = self.loss.forward_pass(y, t)
                print("    -> loss: %f" % l)
                for m in self.metrics:
                    print("    -> %s: %f" % (m.__name__, m(y, t)))
        if plot:
            nDenses = sum([layer.__class__.__name__ == "Dense" for layer in self.layers])
            maximaxi = np.max(np.max([layer.monitor  for layer in self.layers if (layer.__class__.__name__ == "Dense") and layer != None ]))
            plt.clf()
            plt.cla()
            plt.close()
            tmp =4 * int(math.ceil(nDenses / 5.0))
            plt.figure(figsize=(20,tmp), dpi=80)
            plt.subplots_adjust(left=0, right=1, bottom=0, top=1)
            iteri = 1
            for i, layer in enumerate( self.layers):
                if not layer.__class__.__name__ == "Dense":
                    continue
                if layer.plot:
                    plt.subplot(int(math.ceil(nDenses / 5.0)) ,5,iteri)
                    iteri+=1
                    plt.title("Layer "  +str(i))
                    plt.ylim([0, maximaxi])
                    plt.yscale('symlog')
                    plt.bar( [x for x in range(len(layer.monitor))], layer.monitor)
            plt.show()
        
        
    def predict(self, X):
        inps, out = self._forward_pass(X)
        return out

    def _fit_on_batch(self, batch_X, batch_t):
        inps, out = self._forward_pass(batch_X)
        tempo = self.loss.backward_pass(out, batch_t)
        layer_grads = self._backward_pass(inps, tempo)
        grad = self._join(layer_grads)
        deltas = self.optimizer.calculate_deltas(grad)
        j = 0
        for i in xrange(0, len(self.layers), 1):
            if self.layers[i].__class__.__name__ == "Dense":
                self.layers[i].update_weights(deltas[j])
                j+=1


    def _join(self, grads):
        return np.array([g for g in grads if not g is None])

    def _split(self, grads, layer_grads):
        out = []
        start = 0
        for l in layer_grads:
            if l is None:
                out.append(None)
            else:
                out.append(grads[start:start+len(l)])
                start += len(l)
        return out
    
    #Ćwiczenie 2
    def _forward_pass(self, X): 
        inps = []
        output = None
        for layer in self.layers:
            inps.append(X)
            tmp=layer.forward_pass(X)
            X = tmp
        return inps, X
    
    #Ćwiczenie 3
    def _backward_pass(self, inps, grad):
        n = len(self.layers)
        layer_grads = [None for i in xrange(n)]
        weight_grad = [None for i in xrange(n)]
        layer_grads[n-1] = grad
        for i in xrange(n-1, 0,-1):
            input_grad, weights_grad = self.layers[i-1].backward_pass((inps[i-1]), layer_grads[i]  ) 
            weight_grad[i-1] =  weights_grad
            layer_grads[i-1] =  input_grad
 
        return weight_grad

    def _debug_grads(self, X, t):
        layer_grads = []
        for l in self.layers:
            g = l.debug_grad(
                lambda: self.loss.forward_pass(self._forward_pass(X)[1], t)
            )
            if not g is None:
                g = np.array(np.array(g).flat)
            layer_grads.append(g)
        return layer_grads

In [228]:
# Ćwiczenie 4
from sklearn.metrics import accuracy_score, mean_squared_error
def metric(y,t):
    return accuracy_score(np.around(y.flat), t)
def metric2(y,t):
    return mean_squared_error(np.around(y.flat), t)

In [186]:
network = Network(loss=MSE(), optimizer=GD(learning_rate=0.05), metrics=[metric])
network.add(Dense(784,8))
network.add(Dense(8,1)) #jedna warstwa ukryta
network.add(Sigmoid())
network.fit(X_train,y_train, epochs= 5, print_stats=True)
print metric(network.predict(X_test), y_test)

Epoch 1
    -> batch size: 256
    -> loss: 0.025611
    -> metric: 0.955552
Epoch 2
    -> batch size: 256
    -> loss: 0.015795
    -> metric: 0.964459
Epoch 3
    -> batch size: 256
    -> loss: 0.013211
    -> metric: 0.968251
Epoch 4
    -> batch size: 256
    -> loss: 0.012001
    -> metric: 0.970456
Epoch 5
    -> batch size: 256
    -> loss: 0.011259
    -> metric: 0.972308
0.977297297297


In [211]:
#Ćwiczenie 7:1
network1 = Network(loss=Crossentropy(), optimizer=Momentum(alpha=0.01,beta=0.051), metrics=[metric])
network1.add(Dense(784,5))
network1.add(ReLU())
network1.add(Dense(5,1))
network1.add(Sigmoid())
network1.fit(X_train,y_train, epochs= 3, print_stats=True)
print metric(network1.predict(X_train), y_train)

Epoch 1
    -> batch size: 256
    -> loss: 0.358476
    -> metric: 0.870888
Epoch 2
    -> batch size: 256
    -> loss: 0.249654
    -> metric: 0.952112
Epoch 3
    -> batch size: 256
    -> loss: 0.156064
    -> metric: 0.960138
0.96013757827


In [229]:
# Ćwiczenie 7:2
network2 = Network(loss=Crossentropy(), optimizer=Momentum(alpha=0.05,beta=0.25), metrics=[metric])
network2.add(Dense(784,13))
network2.add(ReLU())
network2.add(Dense(13,1))
network2.add(ReLU())
network2.fit(X_train,y_train, epochs= 2, print_stats=True)
print metric(network2.predict(X_test), y_test)

Epoch 1
    -> batch size: 256
    -> loss: nan
    -> metric: 0.000000
Epoch 2
    -> batch size: 256
    -> loss: nan
    -> metric: 0.000000
0.0


Ćwiczenie 7:
Ostatnia warstwa ReLU nie pasuje dobrze do problemy klasyfikacji, gdzie na wyjściu chcemy dostac informację zero jedynkową, lub "procentową".

Ponadto powoduje ona problemy z loss Crossentropy dając NaN.


Epoch 1
    -> batch size: 256
    -> loss: 0.690046
    -> metric: 0.478084
Epoch 2
    -> batch size: 256
    -> loss: 0.677257
    -> metric: 0.523503
Epoch 3
    -> batch size: 256
    -> loss: 0.658492
    -> metric: 0.574654
Epoch 4
    -> batch size: 256
    -> loss: 0.618366
    -> metric: 0.849899
Epoch 5
    -> batch size: 256
    -> loss: 0.538764
    -> metric: 0.915777
Epoch 6
    -> batch size: 256
    -> loss: 0.425359
    -> metric: 0.942058
Epoch 7
    -> batch size: 256
    -> loss: 0.316180
    -> metric: 0.951495
Epoch 8
    -> batch size: 256
    -> loss: 0.230719
    -> metric: 0.958197


0.958918918919


In [214]:
# Ćwiczenie 8:1
network3 = Network(loss=Crossentropy(), optimizer=Momentum(alpha=0.01,beta=0.05), metrics=[metric])
network3.add(Dense(784,32,plot=True))
network3.add(Dense(32,8,plot=True))
network3.add(Sigmoid())
network3.add(Dense(8,1,plot=True))
network3.add(Sigmoid())
network3.fit(X_train,y_train, epochs= 8, print_stats=True,plot=True)

print metric(network3.predict(X_test), y_test)

Epoch 1
    -> batch size: 256
    -> loss: 0.124760
    -> metric: 0.521916
Epoch 2
    -> batch size: 256
    -> loss: 0.124955
    -> metric: 0.521916
Epoch 3
    -> batch size: 256
    -> loss: 0.124796
    -> metric: 0.521916
Epoch 4
    -> batch size: 256
    -> loss: 0.124805
    -> metric: 0.521916
Epoch 5
    -> batch size: 256
    -> loss: 0.124860
    -> metric: 0.521916


0.517837837838


In [236]:
# Ćwiczenie 8:2
network4 = Network(loss=MSE(), optimizer=Momentum(alpha=0.25,beta=0.05), metrics=[metric])
network4.add(Dense(784,8,plot=True))
network4.add(Dense(8,8,plot=True))
network4.add(Sigmoid())
network4.add(Dense(8,8,plot=True))
network4.add(Sigmoid())
network4.add(Dense(8,8,plot=True))
network4.add(Sigmoid())
network4.add(Dense(8,8,plot=True))
network4.add(Sigmoid())
network4.add(Dense(8,8,plot=True))
network4.add(Sigmoid())
network4.add(Dense(8,8,plot=True))
network4.add(Sigmoid())
network4.add(Dense(8,8,plot=True))
network4.add(Sigmoid())
network4.add(Dense(8,8,plot=True))
network4.add(Sigmoid())
network4.add(Dense(8,1,plot=True))
network4.add(Sigmoid())
network4.fit(X_train,y_train, epochs= 5, print_stats=True,plot=True)
print metric(network4.predict(X_test), y_test)

In [218]:
#Ćwiczenie 9
network5 = Network(loss=MSE(), optimizer=Momentum(alpha=0.05,beta=0.25), metrics=[metric])
network5.add(Dense(784,32,init='aaa'))
network5.add(Dense(32,8,init='aaa'))
network5.add(Sigmoid())
network5.add(Dense(8,1,init='aaa'))
network5.add(Sigmoid())
network5.fit(X_train,y_train, epochs= 3, print_stats=True)
print metric(network5.predict(X_test), y_test)


Epoch 1
    -> batch size: 256
    -> loss: 0.239042
    -> metric: 0.521916
Epoch 2
    -> batch size: 256
    -> loss: 0.239042
    -> metric: 0.521916
Epoch 3
    -> batch size: 256
    -> loss: 0.239042
    -> metric: 0.521916
0.517837837838


## Ćwiczenie 1

In [21]:
np.random.seed(43)
d1 = Dense(input_size=4, output_size=3, init="gaussian")
d2 = Dense(input_size=4, output_size=3, init="zeros")
r = ReLU()
s = Sigmoid()
inp = np.random.random(4).reshape((1,-1)) - 0.5
out_grad_4 = np.random.random(4).reshape((1,-1)) - 0.5
out_grad_3 = np.random.random(3).reshape((1,-1)) - 0.5

print "inp:"
print inp.shape
print inp
print "out_grad_4:"
print out_grad_4.shape
print out_grad_4
print "out_grad_3:"
print out_grad_3.shape
print out_grad_3

print "Testing d1..."
print "d1.forward_pass(inp):"
t = d1.forward_pass(inp)
print t.shape
print t
print "d1.backward_pass(inp, out_grad_3):"
t = d1.backward_pass(inp, out_grad_3)
print t[0].shape
print t[0]
print t[1].shape
print t[1]

print "Testing d2..."
print "d2.forward_pass(inp):"
t = d2.forward_pass(inp)
print t.shape
print t
print "d2.backward_pass(inp, out_grad_3):"
t = d2.backward_pass(inp, out_grad_3)
print t[0].shape
print t[0]
print t[1].shape
print t[1]

print "Testing r..."
print "r.forward_pass(inp):"
t = r.forward_pass(inp)
print t.shape
print t
print "r.backward_pass(inp, out_grad_4):"
t = r.backward_pass(inp, out_grad_4)
print t[0].shape
print t[0]
print t[1]

print "Testing s..."
print "s.forward_pass(inp):"
t = s.forward_pass(inp)
print t.shape
print t
print "s.backward_pass(inp, out_grad_4):"
t = s.backward_pass(inp, out_grad_4)
print t[0].shape
print t[0]
print t[1]

inp:
(1L, 4L)
[[-0.05424164  0.16972465 -0.41749995  0.39709858]]
out_grad_4:
(1L, 4L)
[[-0.2019965  -0.23769518 -0.49487045  0.04320252]]
out_grad_3:
(1L, 3L)
[[-0.02440363  0.13637368  0.47820413]]
Testing d1...
d1.forward_pass(inp):
(1L, 3L)
[[-0.02707048 -0.20288487 -0.03403094]]
d1.backward_pass(inp, out_grad_3):
output_grad.shape:  (1L, 3L)
input_grad.shape:  (1L, 4L)
(1L, 4L)
[[-0.01685764  0.21647157  0.04530229  0.03971692]]
(5L, 3L)
[[-0.02440363  0.13637368  0.47820413]
 [ 0.00132369 -0.00739713 -0.02593858]
 [-0.0041419   0.02314597  0.08116303]
 [ 0.01018852 -0.056936   -0.1996502 ]
 [-0.00969065  0.05415379  0.18989418]]
Testing d2...
d2.forward_pass(inp):
(1L, 3L)
[[ 0.  0.  0.]]
d2.backward_pass(inp, out_grad_3):
output_grad.shape:  (1L, 3L)
input_grad.shape:  (1L, 4L)
(1L, 4L)
[[ 0.  0.  0.  0.]]
(5L, 3L)
[[-0.02440363  0.13637368  0.47820413]
 [ 0.00132369 -0.00739713 -0.02593858]
 [-0.0041419   0.02314597  0.08116303]
 [ 0.01018852 -0.056936   -0.1996502 ]
 [-0.00969

## Ćwiczenie 2

In [22]:
np.random.seed(43)
n = Network(loss=MSE(), optimizer=GD(learning_rate=0.001), metrics=[])
n.add(Dense(input_size=4, output_size=3, init="gaussian"))
n.add(ReLU())
n.add(Dense(input_size=3, output_size=4, init="gaussian"))
n.add(ReLU())
n.add(Dense(input_size=4, output_size=1, init="gaussian"))
n.add(Sigmoid())
inp = np.random.random((2,4)) - 0.5
inps, out = n._forward_pass(inp)

print "inp:"
print inp
for i, inp in enumerate(inps):
    print "inps[" + str(i) + "]:"
    print inp
print "out:"
print out

inp:
[[ 0.06066317 -0.08874451  0.22698799 -0.10080311]
 [ 0.17014516  0.20471561  0.10955987  0.04003446]]
inps[0]:
[[ 0.06066317 -0.08874451  0.22698799 -0.10080311]
 [ 0.17014516  0.20471561  0.10955987  0.04003446]]
inps[1]:
[[ 0.06231216  0.          0.        ]
 [ 0.03836232  0.          0.        ]]
inps[2]:
[[ 0.06231216  0.          0.        ]
 [ 0.03836232  0.          0.        ]]
inps[3]:
[[ 0.          0.4840028   0.          0.        ]
 [ 0.          0.48391873  0.          0.        ]]
inps[4]:
[[ 0.          0.4840028   0.          0.        ]
 [ 0.          0.48391873  0.          0.        ]]
inps[5]:
[[ 0.29948146]
 [ 0.29947621]]
out:
[[ 0.57431575]
 [ 0.57431447]]


## Ćwiczenie 3

In [36]:
np.random.seed(43)
n = Network(loss=MSE(), optimizer=GD(learning_rate=0.001), metrics=[])
n.add(Dense(input_size=4, output_size=3, init="gaussian"))
n.add(ReLU())
n.add(Dense(input_size=3, output_size=4, init="gaussian"))
n.add(ReLU())
n.add(Dense(input_size=4, output_size=1, init="gaussian"))
n.add(Sigmoid())

inp = np.random.random((2,4)) - 0.5
target = inp[:,0:1]
inps, out = n._forward_pass(inp)
grad = n.loss.backward_pass(out, target)
layer_grads = n._backward_pass(inps, grad)

print "inp:"
print inp
print "target:"
print target
for i, inp in enumerate(inps):
    print "inps[" + str(i) + "]:"
    print inp
print "out:"
print out
print "grad:"
print grad
for i, grad in enumerate(layer_grads):
    print "layer_grads[" + str(i) + "]:"
    print grad

kosmo test:,  (2L, 3L)
inp:
[[ 0.06066317 -0.08874451  0.22698799 -0.10080311]
 [ 0.17014516  0.20471561  0.10955987  0.04003446]]
target:
[[ 0.06066317]
 [ 0.17014516]]
inps[0]:
[[ 0.06066317 -0.08874451  0.22698799 -0.10080311]
 [ 0.17014516  0.20471561  0.10955987  0.04003446]]
inps[1]:
[[ 1.  0.  0.]
 [ 1.  0.  0.]]
inps[2]:
[[ 1.  0.  0.]
 [ 1.  0.  0.]]
inps[3]:
[[ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]]
inps[4]:
[[ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]]
inps[5]:
[[ 0.29948146]
 [ 0.29947621]]
out:
[[ 0.57431575]
 [ 0.57431447]]
grad:
[[ 0.25682629]
 [ 0.20208465]]
layer_grads[0]:
[[  1.00504857e-04   0.00000000e+00   0.00000000e+00]
 [  1.09423994e-05   0.00000000e+00   0.00000000e+00]
 [  4.06870903e-06   0.00000000e+00   0.00000000e+00]
 [  1.76162598e-05   0.00000000e+00   0.00000000e+00]
 [ -3.89801042e-06   0.00000000e+00   0.00000000e+00]]
layer_grads[1]:
None
layer_grads[2]:
[[ 0.          0.02863277  0.          0.        ]
 [ 0.          0.00148219  0.          0.        ]
 [ 

## Ćwiczenie 5

In [163]:
np.random.seed(43)
ce = Crossentropy()
y = np.random.random(10)
t = np.random.random(10)

print "y:"
print y
print "t:"
print t
print "ce.forward_pass(y,t):"
print ce.forward_pass(y,t)
print "ce.backward_pass(y,t):"
print ce.backward_pass(y,t)

y:
[ 0.11505457  0.60906654  0.13339096  0.24058962  0.32713906  0.85913749
  0.66609021  0.54116221  0.02901382  0.7337483 ]
t:
[ 0.39495002  0.80204712  0.25442113  0.05688494  0.86664864  0.221029
  0.40498945  0.31609647  0.0766627   0.84322469]
ce.forward_pass(y,t):
0.736415962327
ce.backward_pass(y,t):
[-0.27490047 -0.08104869 -0.10469935  0.10054647 -0.24509895  0.5272741
  0.11739401  0.0906406  -0.16913545 -0.05603779]


## Ćwiczenie 6

In [103]:
np.random.seed(43)
opt = Momentum(alpha=0.02, beta=0.99)
grad1 = np.random.random(5)
grad2 = np.random.random(5)
grad3 = np.random.random(5)
opt.calculate_deltas(grad1)
opt.calculate_deltas(grad2)
opt.calculate_deltas(grad3)

print "grad1:"
print grad1
print "grad2:"
print grad2
print "grad3:"
print grad3

print "opt.calculate_deltas(grad1):"
print opt.calculate_deltas(grad1)
print "opt.calculate_deltas(grad2):"
print opt.calculate_deltas(grad2)
print "opt.calculate_deltas(grad3):"
print opt.calculate_deltas(grad3)

grad1:
[ 0.11505457  0.60906654  0.13339096  0.24058962  0.32713906]
grad2:
[ 0.85913749  0.66609021  0.54116221  0.02901382  0.7337483 ]
grad3:
[ 0.39495002  0.80204712  0.25442113  0.05688494  0.86664864]
opt.calculate_deltas(grad1):
[-0.02919466 -0.0529381  -0.0209018  -0.01117572 -0.04443381]
opt.calculate_deltas(grad2):
[-0.04608546 -0.06573052 -0.03151603 -0.01164424 -0.05866444]
opt.calculate_deltas(grad3):
[-0.05352361 -0.08111416 -0.03628929 -0.0126655  -0.07541077]
